In [1]:
import pandas as pd
import pyproj
import geopandas as gpd
import folium
import cv2
import imutils
import numpy as np
import colorsys
import os
import shutil
from os import listdir
import random
from PIL import Image
from PIL import ImageEnhance
import matplotlib.pyplot as plt
import torchvision
from shapely.geometry import Polygon
import math
from PIL import ImageStat
import json

## YOLO preps

### Training data

In [13]:
folder_dir = "./training data/oma street view/suunta1WIDE"
txtlist = []

for x in os.listdir(folder_dir):
        if x.endswith(".txt"):
            txtlist.append(x)

# txtlist = [int(s[:-4]) for s in txtlist]
# txtlist

['0.txt',
 '1.txt',
 '10.txt',
 '100.txt',
 '1000.txt',
 '1001.txt',
 '1002.txt',
 '1003.txt',
 '1004.txt',
 '1005.txt',
 '1006.txt',
 '1007.txt',
 '1008.txt',
 '1009.txt',
 '101.txt',
 '1010.txt',
 '1011.txt',
 '1012.txt',
 '1013.txt',
 '1014.txt',
 '1015.txt',
 '1016.txt',
 '1017.txt',
 '1018.txt',
 '1019.txt',
 '102.txt',
 '1020.txt',
 '1021.txt',
 '1022.txt',
 '1023.txt',
 '1024.txt',
 '1025.txt',
 '1026.txt',
 '1027.txt',
 '1028.txt',
 '1029.txt',
 '103.txt',
 '1030.txt',
 '1031.txt',
 '1032.txt',
 '1033.txt',
 '1034.txt',
 '1035.txt',
 '1036.txt',
 '1037.txt',
 '1038.txt',
 '1039.txt',
 '104.txt',
 '1040.txt',
 '1041.txt',
 '1042.txt',
 '1043.txt',
 '1044.txt',
 '1045.txt',
 '1046.txt',
 '1047.txt',
 '1048.txt',
 '1049.txt',
 '105.txt',
 '1050.txt',
 '1051.txt',
 '1052.txt',
 '1053.txt',
 '1054.txt',
 '1055.txt',
 '1056.txt',
 '1057.txt',
 '1058.txt',
 '1059.txt',
 '106.txt',
 '1060.txt',
 '1061.txt',
 '1062.txt',
 '1063.txt',
 '1064.txt',
 '1065.txt',
 '1066.txt',
 '1067.txt',
 

In [14]:
df_list = (pd.read_csv(folder_dir + "/" + file, sep=" ", header=None, names=["class", "x-value", "y-value", "width", "height"]) for file in txtlist)


manual_label = pd.concat(df_list, ignore_index=True)

manual_label

,class,x-value,y-value,width,height
0,0,0.237120,0.571600,0.003963,0.012034
1,1,0.170850,0.568592,0.004403,0.013237
2,1,0.314619,0.540313,0.003963,0.016847
3,24,0.343008,0.466120,0.008795,0.022951
4,1,0.539850,0.543923,0.007045,0.024067
...,...,...,...,...,...
1913,2,0.637203,0.430601,0.014072,0.043716
1914,23,0.583333,0.550273,0.014512,0.031694
1915,1,0.313738,0.507822,0.011889,0.052948
1916,1,0.522457,0.503610,0.005724,0.030084


In [3]:
classmap = {0:"kolmio",
1:"suojatie",
2:"kävely+pyörä",
3:"parkki_kielletty",
4:"alue_parkki_kielletty",
5:"nuoli_R",
6:"nuoli_L",
7:"nuoli_jakaja",
8:"työmaa",
9:"kielletty_ajosuunta",
10:"R_kielletty_käännös",
11:"L_kielletty_käännös",
12:"yksisuuntainen",
13:"varo_lapsia",
14:"parkki",
15:"liikenneYMPYRÄ",
16:"end_parkki_kielto",
17:"moottori_AJON_kielletty",
18:"pysähtyminen_kielletty",
19:"umpikuja",
20:"pyörätie",
21:"kävelytie",
22:"AJON_kielletty (ympyrä)",
23:"ETUajo-oikeus",
24:"kävely-pyörä_Jaettu",
25:"BUSSIkaista",
26:"STOP"}

In [90]:
list(classmap.values())

['kolmio',
 'suojatie',
 'kävely+pyörä',
 'parkki_kielletty',
 'alue_parkki_kielletty',
 'nuoli_R',
 'nuoli_L',
 'nuoli_jakaja',
 'työmaa',
 'kielletty_ajosuunta',
 'R_kielletty_käännös',
 'L_kielletty_käännös',
 'yksisuuntainen',
 'varo_lapsia',
 'parkki',
 'liikenneYMPYRÄ',
 'end_parkki_kielto',
 'moottori_AJON_kielletty',
 'pysähtyminen_kielletty',
 'umpikuja',
 'pyörätie',
 'kävelytie',
 'AJON_kielletty (ympyrä)',
 'ETUajo-oikeus',
 'kävely-pyörä_Jaettu',
 'BUSSIkaista',
 'STOP']

In [5]:
manual_label.groupby("class")["height"].count().reset_index().sort_values(by="height", ascending=False).rename(columns={"height":"count"}).replace({"class":classmap}).reset_index(drop=True)

,class,count
0,suojatie,504
1,parkki_kielletty,216
2,parkki,173
3,kävely+pyörä,153
4,kolmio,148
5,nuoli_R,106
6,kävely-pyörä_Jaettu,87
7,alue_parkki_kielletty,70
8,työmaa,70
9,moottori_AJON_kielletty,64


In [2]:
swe_labelmap = pd.read_csv("./training data/sweden/train.csv")
swe_labelmap

,file_name,label
0,picture-009375.jpg,PEDESTRIAN_CROSSING
1,picture-511184.jpg,PEDESTRIAN_CROSSING
2,picture-616769.jpg,PEDESTRIAN_CROSSING
3,picture-963518.jpg,PEDESTRIAN_CROSSING
4,picture-486207.jpg,PEDESTRIAN_CROSSING
...,...,...
2498,picture-565027.jpg,PASS_RIGHT_SIDE
2499,picture-034604.jpg,OTHER
2500,picture-258637.jpg,PRIORITY_ROAD
2501,picture-586334.jpg,NO_PARKING


In [3]:
swe_label_other = swe_labelmap[swe_labelmap["label"]=="OTHER"]["file_name"].tolist()
swe_label_other

['picture-761836.jpg',
 'picture-670844.jpg',
 'picture-713895.jpg',
 'picture-251920.jpg',
 'picture-471026.jpg',
 'picture-563907.jpg',
 'picture-688989.jpg',
 'picture-330591.jpg',
 'picture-005603.jpg',
 'picture-462007.jpg',
 'picture-161694.jpg',
 'picture-991761.jpg',
 'picture-224080.jpg',
 'picture-889887.jpg',
 'picture-955993.jpg',
 'picture-841865.jpg',
 'picture-793684.jpg',
 'picture-926224.jpg',
 'picture-671712.jpg',
 'picture-874942.jpg',
 'picture-527844.jpg',
 'picture-616618.jpg',
 'picture-901826.jpg',
 'picture-127544.jpg',
 'picture-299284.jpg',
 'picture-885489.jpg',
 'picture-397764.jpg',
 'picture-427183.jpg',
 'picture-187759.jpg',
 'picture-270522.jpg',
 'picture-506845.jpg',
 'picture-280964.jpg',
 'picture-854312.jpg',
 'picture-051651.jpg',
 'picture-253014.jpg',
 'picture-969100.jpg',
 'picture-318121.jpg',
 'picture-840440.jpg',
 'picture-869651.jpg',
 'picture-633229.jpg',
 'picture-432466.jpg',
 'picture-701122.jpg',
 'picture-026726.jpg',
 'picture-7

In [18]:
# source_folder = "./training data/sweden/data/"
# destination_folder = "./training data/sweden/data_other/"

# for file_name in swe_label_other:
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     shutil.copy(source, destination)

In [47]:
# folder_dir = "./training data/sweden/data_other/"

# for x in os.listdir(folder_dir):
#         if x.endswith(".txt"):
#                 filepath = folder_dir + x
#                 with open(filepath, "r+") as file:
#                         text = file.read()
#                         file.seek(0)
#                         file.truncate()        
#                         file.write(text.split(" ")[0] + " 0.500000 0.500000 1.000000 1.000000")

In [4]:
swe_classmap = {
    "PEDESTRIAN_CROSSING":1,
    "PRIORITY_ROAD":23,
    "PASS_RIGHT_SIDE":5,
    "OTHER":None,
    "30_SIGN":None,
    "110_SIGN":None,
    "70_SIGN":None,
    "50_SIGN":None,
    "60_SIGN":None,
    "100_SIGN":None,
    "80_SIGN":None,
    "90_SIGN":None,
    "PASS_EITHER_SIDE":7,
    "GIVE_WAY":0,
    "STOP":26,
    "NO_PARKING":3,
    "NO_STOPPING_NO_STANDING":18
}

In [5]:
swe_labelmap["label"] = swe_labelmap["label"].map(swe_classmap)
swe_labelmap.dropna(inplace=True)
swe_labelmap["label"] = swe_labelmap["label"].astype(int)
swe_labelmap

,file_name,label
0,picture-009375.jpg,1
1,picture-511184.jpg,1
2,picture-616769.jpg,1
3,picture-963518.jpg,1
4,picture-486207.jpg,1
...,...,...
2496,picture-061352.jpg,5
2498,picture-565027.jpg,5
2500,picture-258637.jpg,23
2501,picture-586334.jpg,3


In [8]:
# folder_dir = "./training data/sweden/data/"

# for index, row in swe_labelmap.iterrows():
#     file_name = row["file_name"][:-4] + ".txt"
#     file = folder_dir + file_name
#     class_num = row["label"]

#     with open(file, "w") as file:
#         file.write(str(class_num) + " 0.500000 0.500000 1.000000 1.000000")

In [8]:
folder_dir = "./training data/cropped_sweden1/data"
txtlist2 = []

for x in os.listdir(folder_dir):
        if x.endswith(".txt"):
            txtlist2.append(x)

# txtlist2 = [int(s[:-4]) for s in txtlist2]

df_list = (pd.read_csv(folder_dir + "/" + file, sep=" ", header=None, names=["class", "x-value", "y-value", "width", "height"]) for file in txtlist2)


manual_label_swe = pd.concat(df_list, ignore_index=True)

manual_label_swe

,class,x-value,y-value,width,height
0,1,0.5,0.5,1.0,1.0
1,15,0.5,0.5,1.0,1.0
2,23,0.5,0.5,1.0,1.0
3,0,0.5,0.5,1.0,1.0
4,5,0.5,0.5,1.0,1.0
...,...,...,...,...,...
1847,23,0.5,0.5,1.0,1.0
1848,1,0.5,0.5,1.0,1.0
1849,5,0.5,0.5,1.0,1.0
1850,1,0.5,0.5,1.0,1.0


In [23]:
manual_label_counts = manual_label.groupby("class")["height"].count().reset_index().sort_values(by="height", ascending=False).rename(columns={"height":"count"}).replace({"class":classmap}).reset_index(drop=True)

In [9]:
manual_label_swe_counts = manual_label_swe.groupby("class")["height"].count().reset_index().sort_values(by="height", ascending=False).rename(columns={"height":"count"}).replace({"class":classmap}).reset_index(drop=True)

In [10]:
manual_label_swe_counts

,class,count
0,suojatie,630
1,ETUajo-oikeus,438
2,nuoli_R,364
3,kolmio,143
4,liikenneYMPYRÄ,88
5,pysähtyminen_kielletty,51
6,parkki_kielletty,45
7,työmaa,21
8,nuoli_jakaja,19
9,STOP,17


In [42]:
label_counts_sum = pd.concat([manual_label_counts, manual_label_swe_counts]).groupby("class").sum().reset_index().sort_values(by="count", ascending=False).reset_index(drop=True)
# label_counts_sum["count"] = label_counts_sum["count"].astype(int)
label_counts_sum

,class,count
0,suojatie,1134
1,nuoli_R,470
2,ETUajo-oikeus,446
3,kolmio,291
4,parkki_kielletty,261
5,parkki,173
6,kävely+pyörä,153
7,liikenneYMPYRÄ,110
8,pysähtyminen_kielletty,101
9,työmaa,91


#### Data splits

In [3]:
# ## MOVE FILES FROM "SUUNTA1WIDE" TO TRAIN-FOLDER 

# folder_dir = "./training data/oma street view/suunta1WIDE/"
# txtlist = []

# for x in os.listdir(folder_dir):
#         if x.endswith(".txt"):
#             if os.path.getsize(folder_dir + x) > 0:
#                 txtlist.append(x)

# txtlist_jpg = [s.replace("txt", "jpg") for s in txtlist]

# source_folder = "./training data/oma street view/suunta1WIDE/"
# destination_folder = "./training data/train/"

# for file_name in txtlist:
#     source = source_folder + file_name
#     destination = destination_folder + "labels/" + file_name
#     shutil.copy(source, destination)

# for file_name in txtlist_jpg:
#     source = source_folder + file_name
#     destination = destination_folder + "images/" + file_name
#     shutil.copy(source, destination)

In [48]:
## CREATE RANDOM SAMPLE LIST

folder_dir = "./training data/train/labels/"
txtlist = []

for x in os.listdir(folder_dir):
        if x.endswith(".txt"):
            if os.path.getsize(folder_dir + x) > 0:
                txtlist.append(x)

val_txtlist = random.sample(txtlist, 200)

In [49]:
# ## MOVE TO VAL-FOLDER

# source_folder_txt = "./training data/train/labels/"
# source_folder_jpg = "./training data/train/images/"
# destination_folder_txt = "./training data/val/labels/"
# destination_folder_jpg = "./training data/val/images/"

# val_txtlist_jpg = [s.replace("txt", "jpg") for s in val_txtlist]

# for file_name in val_txtlist:
#     source = source_folder_txt + file_name
#     destination = destination_folder_txt + file_name
#     shutil.move(source, destination)

# for file_name in val_txtlist_jpg:
#     source = source_folder_jpg + file_name
#     destination = destination_folder_jpg + file_name
#     shutil.move(source, destination)

In [46]:
# ## MOVE TO test-FOLDER

# source_folder_txt = "./training data/train/labels/"
# source_folder_jpg = "./training data/train/images/"
# destination_folder_txt = "./training data/test/labels/"
# destination_folder_jpg = "./training data/test/images/"

# val_txtlist_jpg = [s.replace("txt", "jpg") for s in val_txtlist]

# for file_name in val_txtlist:
#     source = source_folder_txt + file_name
#     destination = destination_folder_txt + file_name
#     shutil.move(source, destination)

# for file_name in val_txtlist_jpg:
#     source = source_folder_jpg + file_name
#     destination = destination_folder_jpg + file_name
#     shutil.move(source, destination)

In [68]:
# folder_dir = "./training data/sweden/data/"
# txtlist = []

# for x in os.listdir(folder_dir):
#         if x.endswith(".txt"):
#             if os.path.getsize(folder_dir + x) > 0:
#                 txtlist.append(x)

###

# txtlist_jpg = [s.replace("txt", "jpg") for s in txtlist]

# source_folder = "./training data/sweden/data/"
# destination_folder = "./training data/train/"

# for file_name in txtlist:
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     shutil.copy(source, destination)

# for file_name in txtlist_jpg:
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     shutil.copy(source, destination)

In [4]:
### NÄYTÄ NIMETTYJEN LIIKENNEMERKKIEN MÄÄRÄ

# folder_dir = "./training data/train/labels/"
folder_dir = "./training data/oma street view/suunta1WIDE/"
txtlist = []

for x in os.listdir(folder_dir):
        if x.endswith(".txt"):
            txtlist.append(x)

df_list = (pd.read_csv(folder_dir + file, sep=" ", header=None, names=["class", "x-value", "y-value", "width", "height"]) for file in txtlist)


manual_label = pd.concat(df_list, ignore_index=True)

manual_label.groupby("class")["height"].count().reset_index().sort_values(by="height", ascending=False).rename(columns={"height":"count"}).replace({"class":classmap}).reset_index(drop=True)

,class,count
0,suojatie,1440
1,parkki_kielletty,440
2,nuoli_R,375
3,parkki,257
4,kolmio,228
5,kävely+pyörä,213
6,kävely-pyörä_Jaettu,130
7,alue_parkki_kielletty,98
8,työmaa,92
9,moottori_AJON_kielletty,90


#### Picture splits

In [ ]:
'''

1. Ota wide kuva ja sen label
2. Katso label tiedostoa rivi riviltä
3. Jokaisen rivin kohdalla:
    a. X koordinaatista katsotaan, mikä (L/C/R) subkuva valitaan
    b. Kopioidaan subkuva uuteen kansioon ja nimetään uudelleen
    c. Lasketaan laatikon koordinaateille uudet arvot
    d. Luodaan tekstitiedosto samalla nimellä omaan kansioon ja liitetään yolo-label tideot

'''

##### LCR

In [224]:
# source_dir = "./training data/oma street view/suunta1WIDE/"
# destination_dir = "./training data/oma street view/"

# for file in os.listdir(source_dir):
#     if file.endswith(".jpg"):
#         kuva = Image.open(source_dir+file)
#         kuvaL = kuva.crop((0, 0, 640, 640))
#         kuvaC = kuva.crop((640, 0, 2*640, 640))
#         kuvaR = kuva.crop((2*640, 0, 3*640, 640))

#         kuvaL.save(f"{destination_dir}suunta1Lwarp/{file}")
#         kuvaC.save(f"{destination_dir}suunta1Cwarp/{file}")
#         kuvaR.save(f"{destination_dir}suunta1Rwarp/{file}")

##### Files

In [73]:
def uudet_koordinaatit(line):

    line = str(line)
    clss = str(int(line.split(" ")[0]))
    x = float(line.split(" ")[1])
    y = float(line.split(" ")[2])
    w = float(line.split(" ")[3])
    h = float(line.split(" ")[4][:-1])

    # x
    x_pix = x*1920
    x2_pix = x_pix % 640
    x2 = x2_pix/640
    
    # L/C/R
    if x_pix <= 640:
        lcr = "L"
    elif x_pix >= 1280:
        lcr = "R"
    else:
        lcr = "C"

    w2 = w*3

    # wfix
    if x2+w2/2 >= 1:  
        over = x2+w2/2-1
        w2 = w2 - over
        x2 = x2-over/2    
        
    if x2-w2 < 0:
        over = abs(x2-w2/2)
        w2 = w2 - over
        x2 = x2+over/2 

    # rounds and return
    x2 = np.round(x2, 6)
    w2 = np.round(w2, 6)

    # class replacements
    # clss = clss.replace("6", "5")
    # clss = clss.replace("11", "10")

    return f"{clss} {x2} {y} {w2} {h}", lcr

In [6]:
# ## LUO UUDET LCR-LABEL TIEDOSTOT

# filedir = "./training data/train/labels/"

# for file in os.listdir(filedir):
#     with open(filedir + file) as file_original:
#         for line in file_original:
#             num = file.split(".")[0]
#             new_coords = uudet_koordinaatit(line)

#             line_new = new_coords[0]
#             lcr = new_coords[1]

#             with open(f"./training data/train/labels/{num}{lcr}.txt", "a") as file_new:
#                 file_new.write(line_new[:-1]+"\n")


In [230]:
### KOPIOI OIKEA LCR KUVA TRAIN-KANSIOON LCR-LABELS MUKAAN

# filedir = "./training data/train/labels/"
# source_folder = "./training data/oma street view/"
# destination_folder = "./training data/train/images/"

# for file in os.listdir(filedir):
#     lcr = file[-5]
#     num = file.split(".")[0][:-1]
#     # print(num)
#     # print(lcr)
    
#     source = f"{source_folder}suunta1{lcr}warp/{num}.jpg"
#     destination = f"{destination_folder}{num}{lcr}.jpg"
#     shutil.copy(source, destination)


In [74]:
# ## MOVE FILES FROM "SUUNTA1WIDE" TO TRAIN-FOLDER SUORAAN LCR-FORMAATISSA

# folder_dir = "./training data/oma street view/suunta1WIDE/"
# txtlist = []

# for x in os.listdir(folder_dir):
#         if x.endswith(".txt"):
#             if os.path.getsize(folder_dir + x) > 0:
#                 txtlist.append(x)

# ##

# for file in txtlist:
#     with open(folder_dir + file) as file_original:
#         for line in file_original:
#             num = file.split(".")[0]
#             new_coords = uudet_koordinaatit(line)

#             line_new = new_coords[0]
#             lcr = new_coords[1]

#             with open(f"./training data/train/labels/{num}{lcr}.txt", "a") as file_new:
#                 file_new.write(line_new+"\n")

# ##

# filedir = "./training data/train/labels/"
# source_folder = "./training data/oma street view/"
# destination_folder = "./training data/train/images/"

# for file in os.listdir(filedir):
#     lcr = file[-5]
#     num = file.split(".")[0][:-1]
#     # print(num)
#     # print(lcr)
    
#     source = f"{source_folder}suunta1{lcr}warp/{num}.jpg"
#     destination = f"{destination_folder}{num}{lcr}.jpg"
#     shutil.copy(source, destination)


### Classmaps

In [5]:
classmap = {0:"kolmio",
1:"suojatie",
2:"kävely+pyörä",
3:"parkki_kielletty",
4:"alue_parkki_kielletty",
5:"nuoli_R",
6:"nuoli_L",
7:"nuoli_jakaja",
8:"työmaa",
9:"kielletty_ajosuunta",
10:"R_kielletty_käännös",
11:"L_kielletty_käännös",
12:"yksisuuntainen",
13:"varo_lapsia",
14:"parkki",
15:"liikenneYMPYRÄ",
16:"end_parkki_kielto",
17:"moottori_AJON_kielletty",
18:"pysähtyminen_kielletty",
19:"umpikuja",
20:"pyörätie",
21:"kävelytie",
22:"AJON_kielletty (ympyrä)",
23:"ETUajo-oikeus",
24:"kävely-pyörä_Jaettu",
25:"BUSSIkaista",
26:"STOP"}

In [6]:
list(classmap.values())

['kolmio',
 'suojatie',
 'kävely+pyörä',
 'parkki_kielletty',
 'alue_parkki_kielletty',
 'nuoli_R',
 'nuoli_L',
 'nuoli_jakaja',
 'työmaa',
 'kielletty_ajosuunta',
 'R_kielletty_käännös',
 'L_kielletty_käännös',
 'yksisuuntainen',
 'varo_lapsia',
 'parkki',
 'liikenneYMPYRÄ',
 'end_parkki_kielto',
 'moottori_AJON_kielletty',
 'pysähtyminen_kielletty',
 'umpikuja',
 'pyörätie',
 'kävelytie',
 'AJON_kielletty (ympyrä)',
 'ETUajo-oikeus',
 'kävely-pyörä_Jaettu',
 'BUSSIkaista',
 'STOP']

In [7]:
classmap_nc27_to_nc17_num = {0: 0,
1: 1,
2: 2,
3: 3,
4: 4,
5: 5,
6: None,
7: None,
8: 6,
9: 7,
10: 8,
11: 9,
12: None,
13: 10,
14: 11,
15: 12,
16: None,
17: 13,
18: 14,
19: None,
20: None,
21: None,
22: 15,
23: 16,
24: 17,
25: None,
26: None}

In [8]:
classmap_nc17 = {
0: "kolmio",
1: "suojatie",
2: "kävely+pyörä",
3: "parkki_kielletty",
4: "alue_parkki_kielletty",
5: "nuoli_R",
6: "työmaa",
7: "kielletty_ajosuunta",
8: "R_kielletty_käännös",
9: "L_kielletty_käännös",
10: "varo_lapsia",
11: "parkki",
12: "liikenneYMPYRÄ",
13: "moottori_AJON_kielletty",
14: "pysähtyminen_kielletty",
15: "AJON_kielletty (ympyrä)",
16: "ETUajo-oikeus",
17: "kävely-pyörä_Jaettu"
}

In [133]:
list(classmap_nc17.values())

['kolmio',
 'suojatie',
 'kävely+pyörä',
 'parkki_kielletty',
 'alue_parkki_kielletty',
 'nuoli_R',
 'työmaa',
 'kielletty_ajosuunta',
 'R_kielletty_käännös',
 'L_kielletty_käännös',
 'varo_lapsia',
 'parkki',
 'liikenneYMPYRÄ',
 'moottori_AJON_kielletty',
 'pysähtyminen_kielletty',
 'AJON_kielletty (ympyrä)',
 'ETUajo-oikeus',
 'kävely-pyörä_Jaettu']

In [81]:
classmap_nc17_inv = {v: k for k, v in classmap_nc17.items()}
classmap_nc17_inv

{'kolmio': 0,
 'suojatie': 1,
 'kävely+pyörä': 2,
 'parkki_kielletty': 3,
 'alue_parkki_kielletty': 4,
 'nuoli_R': 5,
 'työmaa': 6,
 'kielletty_ajosuunta': 7,
 'R_kielletty_käännös': 8,
 'L_kielletty_käännös': 9,
 'varo_lapsia': 10,
 'parkki': 11,
 'liikenneYMPYRÄ': 12,
 'moottori_AJON_kielletty': 13,
 'pysähtyminen_kielletty': 14,
 'AJON_kielletty (ympyrä)': 15,
 'ETUajo-oikeus': 16,
 'kävely-pyörä_Jaettu': 17}

In [82]:
classmap_inv = {v: k for k, v in classmap.items()}
classmap_inv

{'kolmio': 0,
 'suojatie': 1,
 'kävely+pyörä': 2,
 'parkki_kielletty': 3,
 'alue_parkki_kielletty': 4,
 'nuoli_R': 5,
 'nuoli_L': 6,
 'nuoli_jakaja': 7,
 'työmaa': 8,
 'kielletty_ajosuunta': 9,
 'R_kielletty_käännös': 10,
 'L_kielletty_käännös': 11,
 'yksisuuntainen': 12,
 'varo_lapsia': 13,
 'parkki': 14,
 'liikenneYMPYRÄ': 15,
 'end_parkki_kielto': 16,
 'moottori_AJON_kielletty': 17,
 'pysähtyminen_kielletty': 18,
 'umpikuja': 19,
 'pyörätie': 20,
 'kävelytie': 21,
 'AJON_kielletty (ympyrä)': 22,
 'ETUajo-oikeus': 23,
 'kävely-pyörä_Jaettu': 24,
 'BUSSIkaista': 25,
 'STOP': 26}

In [15]:
fin_classmap = {'kolmio':"Väistämisvelvollisuus risteyksessä",
 'suojatie':"Suojatie",
 'kävely+pyörä':"Yhdistetty pyörätie ja jalkakäytävä",
 'parkki_kielletty':"Pysäköinti kielletty",
 'nuoli_R':"Liikenteenjakaja (oikealle)",
 'kielletty_ajosuunta':"Kielletty ajosuunta",
 'R_kielletty_käännös':"Oikealle kääntyminen kielletty",
 'L_kielletty_käännös':"Vasemmalle kääntyminen kielletty",
 'varo_lapsia':"Varo lapsia",
 'parkki':"Parkkipaikka",
 'liikenneYMPYRÄ':"Liikenneympyrä",
 'moottori_AJON_kielletty':"Moottorikäyttöisellä ajoneuvolla ajo kielletty",
 'pysähtyminen_kielletty':"Pysäyttäminen kielletty",
 'AJON_kielletty (ympyrä)':"Ajoneuvolla ajaminen kielletty",
 'ETUajo-oikeus':"Etuajo-oikeutettu tie",
 'kävely-pyörä_Jaettu':"Pyörätie ja jalkakäytävä rinnakkain",
 "alue_parkki_kielletty":"Pysäköintikieltoalue",
 "työmaa":"Työmaa"
 }

### Mapillary data

In [7]:
valid_signs_mapillary_list = ['warning--children--g1',
 'information--pedestrians-crossing--g1',
 'regulatory--no-parking--g1',
 'regulatory--one-way-left--g1',
 'regulatory--keep-left--g1',
 'regulatory--yield--g1',
 'information--parking--g1',
 'regulatory--shared-path-pedestrians-and-bicycles--g1',
 'regulatory--no-entry--g1',
 'regulatory--keep-right--g1',
 'regulatory--dual-path-pedestrians-and-bicycles--g1',
 'regulatory--no-right-turn--g1',
 'regulatory--no-motor-vehicles--g4',
 'regulatory--dual-path-bicycles-and-pedestrians--g1',
 'regulatory--no-left-turn--g2',
 'regulatory--no-left-turn--g1',
 'regulatory--bicycles-only--g1',
 'regulatory--pedestrians-only--g1',
 'regulatory--stop--g1',
 'regulatory--no-stopping--g1',
 'information--parking--g2',
 'regulatory--one-way-right--g1',
 'information--parking--g5',
 'regulatory--no-motor-vehicles--g5',
 'information--dead-end--g1',
 'information--dead-end-except-bicycles--g1',
 'regulatory--one-way-right--g3',
 'regulatory--roundabout--g1',
 'warning--children--g4',
 'information--dead-end-right--g1',
 'regulatory--pass-on-either-side--g1',
 'regulatory--buses-and-taxi-only--g1',
 'regulatory--one-way-straight--g1',
 'information--dead-end-left--g1',
 'regulatory--priority-road--g1',
 'regulatory--road-closed-to-vehicles--g3',
 'information--dead-end-right--g3',
 'regulatory--end-of-no-parking--g1',
 'regulatory--pedestrians-only--g2',
 'regulatory--no-parking--g4',
 'regulatory--one-way-straight--g3',
 'regulatory--pass-on-either-side--g2',
 'regulatory--roundabout--g3',
 'information--dead-end--g3',
 'warning--children--g2',
 'information--parking--g4',
 'regulatory--road-closed-to-vehicles--g1',
 'information--dead-end-left--g2',
 'regulatory--one-way-right--g2',
 'regulatory--no-parking--g3']

In [9]:
classmap_mapillary = {
"regulatory--yield":"kolmio",
"information--pedestrians-crossing":"suojatie",
"regulatory--shared-path-pedestrians-and-bicycles":"kävely+pyörä",
"regulatory--no-parking":"parkki_kielletty",
"regulatory--keep-right":"nuoli_R",
"regulatory--keep-left":"nuoli_L",
"regulatory--pass-on-either-side":"nuoli_jakaja",
"information--parking":"työmaa",
"regulatory--no-entry":"kielletty_ajosuunta",
"regulatory--no-right-turn":"R_kielletty_käännös",
"regulatory--no-left-turn":"L_kielletty_käännös",
"regulatory--one-way-right":"yksisuuntainen",
"regulatory--one-way-straight":"yksisuuntainen",
"regulatory--one-way-left":"yksisuuntainen",
"warning--children":"varo_lapsia",
"information--parking":"parkki",
"information--parking":"parkki",
"information--parking":"parkki",
"regulatory--roundabout":"liikenneYMPYRÄ",
"regulatory--end-of-no-parking":"end_parkki_kielto",
"regulatory--no-motor-vehicles":"moottori_AJON_kielletty",
"regulatory--no-stopping":"pysähtyminen_kielletty",
"information--dead-end":"umpikuja",
"information--dead-end-left":"umpikuja",
"information--dead-end-right":"umpikuja",
"information--dead-end-except-bicycles":"umpikuja",
"regulatory--bicycles-only":"pyörätie",
"regulatory--pedestrians-only":"kävelytie",
"regulatory--road-closed-to-vehicles":"AJON_kielletty (ympyrä)",
"regulatory--priority-road":"ETUajo-oikeus",
"regulatory--dual-path-bicycles-and-pedestrians":"kävely-pyörä_Jaettu",
"regulatory--dual-path-pedestrians-and-bicycles":"kävely-pyörä_Jaettu",
"regulatory--buses-and-taxi-only":"BUSSIkaista",
"regulatory--stop":"STOP"
}

In [11]:
json_fol = "./training data/mapillary/mtsd_v2_fully_annotated/annotations/"
label_fol = "./training data/mapillary/labels/"

keep_files_mapillary_list = []

for file in os.listdir(json_fol):
    with open(json_fol+file) as jsonfile:
        data = json.load(jsonfile)
        for object in data["objects"]:
            label = object["label"]
            
            if any(label in s for s in valid_signs_mapillary_list):
                keep_files_mapillary_list.append(file)

In [12]:
keep_files_mapillary_list = set(keep_files_mapillary_list)

In [26]:
def bbox_coords_to_yolo(x1, y1, x2, y2, im_w, im_h):
    x = np.round(np.mean((x1,x2))/im_w,6)
    y = np.round(np.mean((y1,y2))/im_h,6)

    w = np.round(abs((x2-x1)/im_w),6)
    h = np.round(abs((y2-y1)/im_h),6)


    return x, y, w, h

In [14]:
json_fol = "./training data/mapillary/mtsd_v2_fully_annotated/annotations/"
# file = "__5wSiAMUy7m9ssKZZ2cEA.json"
label_fol = "./training data/mapillary/labels_full/"

for file in keep_files_mapillary_list:
    with open(json_fol+file) as jsonfile:
        txtfile = file.replace("json", "txt")
        data = json.load(jsonfile)

        if data["ispano"] == False:
            width = int(data["width"])
            height = int(data["height"])
            
            for object in data["objects"]:
                label = object["label"] 

                if any(label in s for s in valid_signs_mapillary_list):
                    bbox = object["bbox"]
                    x1 = int(bbox["xmin"])
                    x2 = int(bbox["xmax"])
                    y1 = int(bbox["ymin"])
                    y2 = int(bbox["ymax"])

                    x, y, w, h = bbox_coords_to_yolo(x1, y1, x2, y2, width, height)

                    try:
                        clss = classmap_mapillary[label[:-4]]
                        clss_num = classmap_inv[clss]

                        line = f"{clss_num} {x} {y} {w} {h}"

                        with open(label_fol+txtfile, "a") as labelfile:
                            labelfile.write(line+"\n")

                    except:
                        None


In [15]:
label_fol = "./training data/mapillary/labels/"

source_folder = "./training data/mapillary/images_all/"
destination_folder = "./training data/mapillary/images/"

for filename in os.listdir(label_fol):
    name = filename[:-4]
    
    source = source_folder + name + ".jpg"
    destination = destination_folder + name + ".jpg"
    shutil.copy(source, destination)

In [16]:
def uudet_koordinaatit_mapillary_1280_640(line, h_kerroin):
    
    line = str(line)
    clss = str(int(line.split(" ")[0]))
    x = float(line.split(" ")[1])
    y = float(line.split(" ")[2])
    w = float(line.split(" ")[3])
    h = float(line.split(" ")[4][:-1]) * h_kerroin

    y2 = h_kerroin*y-h_kerroin/2+0.5

    new_line = f"{clss} {x} {y2} {w} {h}"

    return clss, x, y2, w, h

In [17]:
image_folder_in = "./training data/mapillary/images_all/"
image_folder_out = "./training data/mapillary/images/"
label_fol_in = "./training data/mapillary/labels_full/"
label_fol_out = "./training data/mapillary/labels/"


for file_txt in os.listdir(label_fol_in):
    try:
        file_jpg = file_txt.replace("txt", "jpg")

        img = Image.open(image_folder_in+file_jpg)

        w_pix = img.size[0]
        h_pix = img.size[1]

        new_h_pix = int(w_pix/2)

        h_diff = h_pix - new_h_pix
        h_kerroin = h_pix/new_h_pix

        upper = h_pix - h_diff/2
        lower = h_diff/2

        img2 = img.crop((0, lower, w_pix, upper))
        img2 = img2.resize((1280, 640))

        with open(label_fol_in+file_txt) as label_file:
            for line in label_file:
                clss, x, y, w, h = uudet_koordinaatit_mapillary_1280_640(line, h_kerroin)

                if y>0:
                        # yfix
                    if y+h/2 >= 1:  
                        over = y+h/2-1
                        h = h - over
                        y = y-over/2    
                        
                    if y-h < 0:
                        over = abs(y-h/2)
                        h = h - over
                        y = y+over/2 

                    new_line = f"{clss} {x} {y} {w} {h}"

                    with open(label_fol_out+file_txt, "a") as label_file_out:
                        label_file_out.write(new_line+"\n")

        img2.save(image_folder_out+file_jpg)
    except:
        None

#### LCR

In [20]:
source_dir = "./training data/mapillary/images/"
destination_dir = "./training data/mapillary/SQUARE/images/"

for file in os.listdir(source_dir):
    kuva = Image.open(source_dir+file)
    kuvaL = kuva.crop((0, 0, 640, 640))
    kuvaR = kuva.crop((640, 0, 2*640, 640))

    kuvaL.save(f"{destination_dir}{file[:-4]}_L.jpg")
    kuvaR.save(f"{destination_dir}{file[:-4]}_R.jpg")

In [ ]:
def uudet_koordinaatit_mapillary(line):

    line = str(line)
    clss = str(int(line.split(" ")[0]))
    x = float(line.split(" ")[1])
    y = float(line.split(" ")[2])
    w = float(line.split(" ")[3])
    h = float(line.split(" ")[4][:-1])

    # x
    x_pix = x*1280
    x2_pix = x_pix % 640
    x2 = x2_pix/640
    
    # L/C/R
    if x_pix <= 640:
        lcr = "L"
    else:
        lcr = "R"

    w2 = w*2

    # wfix
    if x2+w2/2 >= 1:  
        over = x2+w2/2-1
        w2 = w2 - over
        x2 = x2-over/2    
        
    if x2-w2 < 0:
        over = abs(x2-w2/2)
        w2 = w2 - over
        x2 = x2+over/2 

    # rounds and return
    x2 = np.round(x2, 6)
    w2 = np.round(w2, 6)

    return f"{clss} {x2} {y} {w2} {h}", lcr

In [ ]:
## LUO UUDET LCR-LABEL TIEDOSTOT

source_dir = "./training data/mapillary/labels/"
destination_dir = "./training data/mapillary/SQUARE/labels/"

for file in os.listdir(source_dir):
    with open(source_dir + file) as file_original:
        for line in file_original:
            id = file.split(".")[0]
            new_coords = uudet_koordinaatit_mapillary(line)

            line_new = new_coords[0]
            lcr = new_coords[1]

            with open(f"{destination_dir}{id}_{lcr}.txt", "a") as file_new:
                file_new.write(line_new[:-1]+"\n")

In [ ]:
label_fol = "./training data/mapillary/SQUARE/labels/"

source_folder = "./training data/mapillary/images_LR/"
destination_folder = "./training data/mapillary/SQUARE/images/"

for filename in os.listdir(label_fol):
    name = filename[:-4]
    
    source = source_folder + name + ".jpg"
    destination = destination_folder + name + ".jpg"
    shutil.copy(source, destination)

#### Val

In [2]:
# train_labels = "./Yolo_project/yolov7-custom/data/train/labels/"
# train_images = "./Yolo_project/yolov7-custom/data/train/images/"

# val_labels = "./Yolo_project/yolov7-custom/data/val/labels/"
# val_images = "./Yolo_project/yolov7-custom/data/val/images/"

# txtlist = os.listdir(train_labels)
# val_txtlist = random.sample(txtlist, 1200)
# val_jpglist = (s.replace("txt", "jpg") for s in val_txtlist)

# for file_name in val_txtlist:
#     source = train_labels + file_name
#     destination = val_labels + file_name
#     shutil.move(source, destination)

# for file_name in val_jpglist:
#     source = train_images + file_name
#     destination = val_images + file_name
#     shutil.move(source, destination)

### Sweden2

In [34]:
classmap_swe2_nc27 = {'100_SIGN':None,
 '110_SIGN':None,
 '120_SIGN':None,
 '30_SIGN':None,
 '50_SIGN':None,
 '60_SIGN':None,
 '70_SIGN':None,
 '80_SIGN':None,
 '90_SIGN':None,
 'GIVE_WAY':0,
 'NO_PARKING':3,
 'NO_STOPPING_NO_STANDING':18,
 'OTHER':None,
 'PASS_EITHER_SIDE':7,
 'PASS_LEFT_SIDE':6,
 'PASS_RIGHT_SIDE':5,
 'PEDESTRIAN_CROSSING':1,
 'PRIORITY_ROAD':23,
 'STOP':26,
 'URDBL':None}

In [37]:
label_file_path = "./training data/sweden2/set1_annotations.txt"
label_fol = "./training data/sweden2/labels/"



with open(label_file_path) as file:
    for line in file:
        name = line.split(":")[0]
        signs = line.split(":")[1]

        for sign in signs.split(";"):
            if len(sign)>12:
                x1 = float(sign.split(",")[1].strip())
                y1 = float(sign.split(",")[2].strip())
                x2 = float(sign.split(",")[3].strip())
                y2 = float(sign.split(",")[4].strip())
                clss = sign.split(",")[6].strip()
                clss_num = classmap_swe2_nc27[clss]

                x, y, w, h = bbox_coords_to_yolo(x1, y1, x2, y2, 1280, 960)

                with open(label_fol+name.replace("jpg", "txt"), "a") as new_label_file:
                    new_label_file.write(f"{clss_num} {x} {y} {w} {h}")

                # print(f"{name} {clss_num} {x} {y} {w} {h}")



### Oma 1C_shift

In [40]:
source_folder = "./images/fov18/suunta1C_shift/"
destination_folder = "./training data/oma street view/suunta1C_shift/labels/"

for file_name in os.listdir(source_folder):
    id = file_name[:-4]

    source = source_folder + id +".txt"
    destination = destination_folder + id +"_s1C_shift.txt"
    # shutil.copy(source, destination)

In [28]:
source_dir = "./training data/oma street view/suunta1C_shift/"

txtlist = []

for x in os.listdir(source_dir):
        if x.endswith(".txt"):
            if os.path.getsize(source_dir + x) > 0:
                txtlist.append(x)

jpglist = [s.replace("txt", "jpg") for s in txtlist]


In [31]:
# source_folder = "./training data/oma street view/suunta1C_shift/"
# destination_folder = "./training data/train/labels/"

# for file in txtlist:
#     source = source_folder + file
#     destination = destination_folder + file
#     shutil.copy(source, destination)

### BG

In [11]:
train_fol = "./training data/train/images/"
val_fol = "./training data/val/images/"
test_fol = "./training data/test/images/"

labelled_images = []

for fol in [train_fol, val_fol, test_fol]:
    for x in os.listdir(fol):
            labelled_images.append(x)

labelled_images

['1007L.jpg',
 '100L.jpg',
 '1010C.jpg',
 '1010R.jpg',
 '1014C.jpg',
 '1014R.jpg',
 '1017R.jpg',
 '1019C.jpg',
 '101C.jpg',
 '1023C.jpg',
 '1023R.jpg',
 '1024L.jpg',
 '1026L.jpg',
 '1027C.jpg',
 '102C.jpg',
 '1031C.jpg',
 '1032C.jpg',
 '1033R.jpg',
 '1036C.jpg',
 '103C.jpg',
 '1044C.jpg',
 '1044L.jpg',
 '1044R.jpg',
 '1045C.jpg',
 '1046C.jpg',
 '1049L.jpg',
 '104R.jpg',
 '1050R.jpg',
 '1052R.jpg',
 '1053C.jpg',
 '1055C.jpg',
 '1062R.jpg',
 '1065C.jpg',
 '1068C.jpg',
 '1070C.jpg',
 '1073C.jpg',
 '1075R.jpg',
 '1079C.jpg',
 '1083C.jpg',
 '1083R.jpg',
 '1086C.jpg',
 '1087C.jpg',
 '1087R.jpg',
 '1089R.jpg',
 '1091L.jpg',
 '1092L.jpg',
 '1092R.jpg',
 '1093L.jpg',
 '1095R.jpg',
 '1096C.jpg',
 '1098C.jpg',
 '1098R.jpg',
 '1101C.jpg',
 '1102L.jpg',
 '1102R.jpg',
 '1108C.jpg',
 '110R.jpg',
 '1113C.jpg',
 '1114C.jpg',
 '1114L.jpg',
 '1114R.jpg',
 '1116C.jpg',
 '111L.jpg',
 '1122L.jpg',
 '1122R.jpg',
 '1124C.jpg',
 '1124L.jpg',
 '1124R.jpg',
 '1127L.jpg',
 '1128C.jpg',
 '1129C.jpg',
 '112R.jpg',


In [12]:
len(labelled_images)

2071

In [26]:
### Tee bg-kuva -kansio

# C_fol = "./training data/oma street view/suunta1Cwarp/"
# L_fol = "./training data/oma street view/suunta1Lwarp/"
# R_fol = "./training data/oma street view/suunta1Rwarp/"

# for fol in [C_fol, L_fol, R_fol]:
#     for x in os.listdir(fol):
#         if x not in labelled_images:
#             num = x.split(".")[0]
#             source = fol + x
#             destination = "./training data/oma street view/bg/" + num + fol[-6] + ".jpg"

#             shutil.copy(source, destination)

### Fix class imbalance

In [13]:
# train_label_fol = "./training data/train/labels/"
train_label_fol = "./Yolo_project/yolov7-custom/data/own data/train/labels/"


train_labels_df_rows_list = (pd.read_csv(train_label_fol + file, sep=" ", header=None, names=["class", "x-value", "y-value", "width", "height"]) for file in os.listdir(train_label_fol) if "comp" not in file)

train_labels_df = pd.concat(train_labels_df_rows_list, ignore_index=True)

train_labels_df

,class,x-value,y-value,width,height
0,3,0.769969,0.499584,0.096006,0.098088
1,1,0.878082,0.426316,0.021126,0.037895
2,0,0.711360,0.571600,0.011889,0.012034
3,1,0.512550,0.568592,0.013209,0.013237
4,1,0.943857,0.540313,0.011889,0.016847
...,...,...,...,...,...
4697,5,0.057879,0.860526,0.115759,0.182105
4698,10,0.164173,0.567368,0.087147,0.069474
4699,1,0.941214,0.507822,0.035667,0.052948
4700,11,0.975561,0.498797,0.025098,0.039711


In [18]:
list = []
for file in os.listdir(train_label_fol):
    if "comp" not in file:
        list.append(file)
print(len(list))

2514


In [11]:
# for file in os.listdir(train_label_fol):
#     print(file)
#     with open(train_label_fol+file) as text:
#         print(text.read())


In [14]:
# delete_classes_list = [6,7,12,16,19,20,21,25,26]
train_labels_df["class"] = train_labels_df["class"].astype(int)
# train_labels_df = train_labels_df[~train_labels_df["class"].isin(delete_classes_list)]

labelcounts_original_data = train_labels_df.groupby("class")["width"].count().reset_index().rename(columns={"width":"count"})
labelcounts_original_data

,class,count
0,0,296
1,1,1943
2,2,241
3,3,575
4,4,120
5,5,506
6,6,125
7,7,67
8,8,21
9,9,34


In [16]:
df = labelcounts_original_data.copy()
df["class"] = df["class"].map(classmap_nc17)
df["class"] = df["class"].map(fin_classmap)
# df.sum()
df

,class,count
0,Väistämisvelvollisuus risteyksessä,296
1,Suojatie,1943
2,Yhdistetty pyörätie ja jalkakäytävä,241
3,Pysäköinti kielletty,575
4,Pysäköintikieltoalue,120
5,Liikenteenjakaja (oikealle),506
6,Työmaa,125
7,Kielletty ajosuunta,67
8,Oikealle kääntyminen kielletty,21
9,Vasemmalle kääntyminen kielletty,34


In [121]:
labelcounts_synthetic_need = labelcounts_original_data.copy()
labelcounts_synthetic_need["count"] = (700 - labelcounts_synthetic_need["count"]) / 2.5
labelcounts_synthetic_need["count"] = labelcounts_synthetic_need["count"].astype(int)
# labelcounts_synthetic_need

In [122]:
omacomp_label_dir = "./training data/oma street view/cropped/labels/"
swecomp_label_dir = "./training data/cropped_sweden1/data/labels/"

file_list_omacomp = [file for file in os.listdir(omacomp_label_dir) if os.path.getsize(omacomp_label_dir + file) > 0] 
file_list_swecomp = [file for file in os.listdir(swecomp_label_dir) if os.path.getsize(swecomp_label_dir + file) > 0] 

file_names = []
file_contents = []

for file_name in file_list_omacomp:
    with open(omacomp_label_dir + file_name, "r") as file:
        content = file.read().split(" ")[0]

    file_names.append(file_name)
    file_contents.append(content)

for file_name in file_list_swecomp:
    with open(swecomp_label_dir + file_name, "r") as file:
        content = file.read().split(" ")[0]

    file_names.append(file_name)
    file_contents.append(content)

data = {
    'filename': file_names,
    'class': file_contents
}

comp_labels_filename_class_df = pd.DataFrame(data)

# comp_labels_filename_class_df

In [123]:
comp_class_count_df = comp_labels_filename_class_df.groupby("class")["filename"].count().reset_index().rename(columns={"filename":"count"})
comp_class_count_df["class"] = comp_class_count_df["class"].astype(int)
comp_class_count_df = comp_class_count_df.sort_values(by="class").reset_index(drop=True)
# comp_class_count_df

In [124]:
# labelcounts_synthetic_need["class"] = labelcounts_synthetic_need["class"].astype(int)
# comp_class_count_df["class"] = comp_class_count_df["class"].astype(int)

In [125]:
comp_class_count_df = comp_class_count_df.merge(labelcounts_synthetic_need, on="class", suffixes=("", "_need"))
comp_class_count_df["x"] = comp_class_count_df["count_need"] / comp_class_count_df["count"]
comp_class_count_df["x"] = comp_class_count_df["x"].apply(lambda x: math.ceil(x))
# comp_class_count_df

In [126]:
groups = comp_labels_filename_class_df.groupby("class")

oversampled_dfs = []

for clss, group in groups:
    x_x_group = []
    clss = int(clss)
    x = comp_class_count_df.loc[clss, "x"]
    need = comp_class_count_df.loc[clss, "count_need"]

    if need > 0:
        for i in range(x):
            x_x_group.append(group)
            x_x_group_df = pd.concat(x_x_group)
            
        x_x_group_df = x_x_group_df.head(need)
        oversampled_dfs.append(x_x_group_df)

final_comp_class_df = pd.concat(oversampled_dfs).reset_index(drop=True)

In [127]:
final_comp_class_df = final_comp_class_df.sample(frac=1).reset_index(drop=True)
# final_comp_class_df

In [128]:
delete_classes_list = [6,7,12,16,19,20,21,25,26]
final_comp_class_df["class"] = final_comp_class_df["class"].astype(int)
final_comp_class_df = final_comp_class_df[~final_comp_class_df["class"].isin(delete_classes_list)]

In [129]:
final_comp_class_df.groupby("class").count()

,filename
class,
0,161
2,183
3,50
4,232
5,77
8,230
9,253
10,271
11,266


In [130]:
signlist_txt = final_comp_class_df["filename"].tolist()

In [131]:
len(signlist_txt)

3650

### Swe composits

In [58]:
# folder_sign_txt = "./training data/cropped_sweden1/data/labels/"

# signlist_txt = []

# for file in os.listdir(folder_sign_txt):
#     signlist_txt.append(file)

# signlist_txt

In [59]:
def bbox_yolo_to_coords(x,y,w,h):
    y = 1-y

    x1 = round((x*640)-(w*640/2))
    x2 = round((x*640)+(w*640/2))

    y1 = round((y*640)+(h*640/2))
    y2 = round((y*640)-(h*640/2))

    return x1, y1, x2, y2

In [60]:
bbox_yolo_to_coords(0.770312, 0.624219, 0.262500, 0.279687)

(409, 330, 577, 151)

In [61]:
def intersection(bbox1, bbox2):
    x1a, y1a, x2a, y2a = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x1b, y1b, x2b, y2b = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    polygon_a = Polygon([[x1a, y1a], [x2a, y1a], [x2a, y2a], [x1a, y2a]])
 
    polygon_b = Polygon([[x1b, y1b], [x2b, y1b], [x2b, y2b], [x1b, y2b]])

    intersection = polygon_a.intersection(polygon_b).area/polygon_a.union(polygon_b).area

    return intersection

In [62]:
def rotate_image_array(img, angle):
    size_reverse = np.array(img.shape[1::-1]) # swap x with y
    M = cv2.getRotationMatrix2D(tuple(size_reverse / 2.), angle, 1.)
    MM = np.absolute(M[:,:2])
    size_new = MM @ size_reverse
    M[:,-1] += (size_new - size_reverse) / 2.
    return cv2.warpAffine(img, M, tuple(size_new.astype(int)))

In [63]:
def pyöritä_kuvaa(kuva_in, kulma):
    kuva = np.array(kuva_in)
    kuva = cv2.cvtColor(kuva, cv2.COLOR_RGB2BGRA)

    kuvar = rotate_image_array(kuva, kulma)

    color_converted = cv2.cvtColor(kuvar, cv2.COLOR_BGRA2RGBA)
    pil_image=Image.fromarray(color_converted).convert("RGBA")

    return pil_image

In [64]:
def shear_x(kuva_in, shear):
    original_image = np.array(kuva_in)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_RGBA2BGRA)

    if shear<0: original_image = cv2.flip(original_image, 1)

    rows, cols, ch = original_image.shape

    bgr_channels = original_image[:, :, :3]
    alpha_channel = original_image[:, :, 3]

    perspective_matrix = np.float32([[1, abs(shear), 0], [0, 1, 0], [0, 0, 1]])

    w = int(cols*(1+abs(shear)))
    h = int(rows)

    transformed_bgr = cv2.warpPerspective(bgr_channels, perspective_matrix, (w, h))
    transformed_alpha = cv2.warpPerspective(alpha_channel, perspective_matrix, (w, h))

    transformed_image = np.dstack((transformed_bgr, transformed_alpha))

    if shear<0: transformed_image = cv2.flip(transformed_image, 1)

    color_converted = cv2.cvtColor(transformed_image, cv2.COLOR_BGRA2RGBA)
    pil_image=Image.fromarray(color_converted).convert("RGBA")

    return pil_image

In [65]:
def shear_y(kuva_in, shear):
    original_image = np.array(kuva_in)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_RGBA2BGRA)

    if shear<0: original_image = cv2.flip(original_image, 1)

    rows, cols, ch = original_image.shape

    bgr_channels = original_image[:, :, :3]
    alpha_channel = original_image[:, :, 3]

    perspective_matrix = np.float32([[1, 0, 0], [abs(shear), 1, 0], [0, 0, 1]])

    w = int(cols)
    h = int(rows*(1+abs(shear)))

    transformed_bgr = cv2.warpPerspective(bgr_channels, perspective_matrix, (w, h))
    transformed_alpha = cv2.warpPerspective(alpha_channel, perspective_matrix, (w, h))

    transformed_image = np.dstack((transformed_bgr, transformed_alpha))

    if shear<0: transformed_image = cv2.flip(transformed_image, 1)

    color_converted = cv2.cvtColor(transformed_image, cv2.COLOR_BGRA2RGBA)
    pil_image=Image.fromarray(color_converted).convert("RGBA")

    return pil_image

In [66]:
def brightness(im):
   stat = ImageStat.Stat(im)
   return stat.mean[0]

In [68]:
# # folder_sign_txt = "./training data/cropped_sweden1/data/labels/"
# folder_sign_txt = "./training data/oma street view/cropped/labels/"

# signlist_txt = []

# for file in os.listdir(folder_sign_txt):
#     signlist_txt.append(file)

# signlist_txt

['omacomp_1001.txt',
 'omacomp_1001_s1C_shift1.txt',
 'omacomp_1002.txt',
 'omacomp_1003.txt',
 'omacomp_10031.txt',
 'omacomp_10071.txt',
 'omacomp_10072.txt',
 'omacomp_10073.txt',
 'omacomp_10074.txt',
 'omacomp_10075.txt',
 'omacomp_10076.txt',
 'omacomp_10077.txt',
 'omacomp_10078.txt',
 'omacomp_10079.txt',
 'omacomp_10101.txt',
 'omacomp_10102.txt',
 'omacomp_10103.txt',
 'omacomp_1011.txt',
 'omacomp_10110.txt',
 'omacomp_1012.txt',
 'omacomp_1012_s1C_shift1.txt',
 'omacomp_1012_s1C_shift2.txt',
 'omacomp_1012_s1C_shift3.txt',
 'omacomp_1012_s1C_shift4.txt',
 'omacomp_1013.txt',
 'omacomp_1014.txt',
 'omacomp_10141.txt',
 'omacomp_10142.txt',
 'omacomp_10143.txt',
 'omacomp_10144.txt',
 'omacomp_1015.txt',
 'omacomp_1015_s1C_shift1.txt',
 'omacomp_1015_s1C_shift2.txt',
 'omacomp_1016.txt',
 'omacomp_1017.txt',
 'omacomp_10171.txt',
 'omacomp_10172.txt',
 'omacomp_1018.txt',
 'omacomp_1019.txt',
 'omacomp_10191.txt',
 'omacomp_10192.txt',
 'omacomp_1020_s1C_shift1.txt',
 'omacom

In [70]:
# folder_sign = "./training data/cropped_sweden1/data/images/"
# folder_sign_txt = "./training data/cropped_sweden1/data/labels/"
# folder_bg = "./training data/oma street view/bg/copypaste_bg/"
# folder_export_jpg = "./training data/cropped_sweden1/composite/images/"
# folder_export_txt = "./training data/cropped_sweden1/composite/labels/"

# folder_sign = "./training data/oma street view/cropped/images/"
# folder_sign_txt = "./training data/oma street view/cropped/labels/"
# folder_bg = "./training data/oma street view/bg/copypaste_bg/"
# folder_export_jpg = "./training data/oma street view/composite/images/"
# folder_export_txt = "./training data/oma street view/composite/labels/"

folder_sign = "./training data/cropped_all/images/"
folder_sign_txt = "./training data/cropped_all/labels/"
folder_bg = "./training data/oma street view/bg/copypaste_bg/"
folder_export_jpg = "./training data/train/images/"
folder_export_txt = "./training data/train/labels/"

def luo_composite(i):
    bgjpg = random.choice(os.listdir(folder_bg))
    img1 = Image.open(folder_bg + bgjpg)
    bboxlist = []

    for x in range(3):
        # signtxt = random.choice(os.listdir(folder_sign_txt))
        signtxt = signlist_txt[i+x]
        signjpg = signtxt.replace("txt", "jpg")

        img2 = Image.open(folder_sign + signjpg).convert("RGBA")

        # Korjaa sign (img2) samalle kirkkaudelle kuin bg (img1) - COLOR
        br_bg = brightness(img1)
        br_sign = brightness(img2)
        br_fac = br_bg/br_sign

        enhancer = ImageEnhance.Brightness(img2)
        img2 = enhancer.enhance(br_fac)

        # DATA TRANSFORMATION - shear
        if np.random.randint(0,4)==0:
            img2 = shear_x(img2, np.random.randint(-20,20)/100)

        if np.random.randint(0,4)==0:
            img2 = shear_y(img2, np.random.randint(-20,20)/100)

        # leveys ja korkeus
        w = img2.size[0]
        h = img2.size[1]

        # DATA TRANSFORMATION - scale
        sw = np.random.randint(8,20)/10
        sh = sw

        if np.random.randint(0,6)==0:
            sw = sw* np.random.randint(5,9)/10
        
        w2 = round(w*sw)
        h2 = round(h*sh)

        try:
            x = np.random.randint(w2,640-w2)
            y = np.random.randint(h2+100,640-h2)
        except:
            try:
                sw = 1
                sh = 1
                w2 = round(w*sw)
                h2 = round(h*sh)

                x = np.random.randint(w2,640-w2)
                y = np.random.randint(h2,640-h2)

            except:
                try:
                    sw = 0.5
                    sh = 0.5
                    w2 = round(w*sw)
                    h2 = round(h*sh)

                    x = np.random.randint(w2,640-w2)
                    y = np.random.randint(h2,640-h2)
                except:
                    pass


        # DATA TRANSFORMATION - rotation
        if np.random.randint(0,6)==0:
            img2 = pyöritä_kuvaa(img2, np.random.randint(-10,10))

        #coordinate calcs
        xc = x+(w2/2)
        yc = y+(h2/2)

        xcd = format(xc/640, ".6f")
        ycd = format(yc/640, ".6f")

        wd = format(w2/640, ".6f")
        hd = format(h2/640, ".6f")

        x1, y1, x2, y2 = bbox_yolo_to_coords(float(xcd), float(ycd), float(wd), float(hd))
        
        bbox0 = [x1, y1, x2, y2]
        bboxlist.append([x1, y1, x2, y2])

        # bboxcheck, kuvan liittäminen ja uuden bbox infon kirjaaminen
        bboxcheck = 0
        if x>1:
            for bbox in bboxlist[:-1]:
                bboxcheck += intersection(bbox0, bbox)
        
        if x1<640 and y1<640 and x2<640 and y2<640 and bboxcheck==0:
            img2 = img2.resize((w2, h2))
            img1.paste(img2, (x,y), img2)

            #txtprint
            with open(folder_sign_txt+signtxt) as sign_text: # lue class
                clss = sign_text.read().split(" ")[0]

            line_new = f"{clss} {xcd} {ycd} {wd} {hd}" # muodosta uusi line

            with open(f"{folder_export_txt}{i}_comp.txt", "a") as file_new: # printtaa uusi line
                file_new.write(line_new+"\n")

    img1.save(folder_export_jpg + f"{i}_comp.jpg")


In [132]:
for i in range(len(signlist_txt)-3):
    # luo_composite(i)

# for i in range(200):
    # luo_composite(i)

c:\Users\pyryp\anaconda3\envs\geoenv\Lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


### Omat composits

In [52]:
def bbox_yolo_to_coords_wide(x,y,w,h):
    y = 1-y

    x1 = round((x*1920)-(w*1920/2))
    x2 = round((x*1920)+(w*1920/2))

    y1 = round((y*640)+(h*640/2))
    y2 = round((y*640)-(h*640/2))

    return x1, y1, x2, y2

In [59]:
def bbox_yolo_to_coords_wide(x,y,w,h):
    y = 1-y

    x1 = round((x*640)-(w*640/2))
    x2 = round((x*640)+(w*640/2))

    y1 = round((y*640)+(h*640/2))
    y2 = round((y*640)-(h*640/2))

    return x1, y1, x2, y2

In [60]:
folder_sign = "./training data/oma street view/suunta1C_shift/"
folder_export = "./training data/oma street view/cropped/"

txtlist = []

for file in os.listdir(folder_sign):
    if file.endswith(".txt"):
        if os.path.getsize(folder_sign + file) > 0:
            txtlist.append(file)

txtlist_jpg = [s.replace("txt", "jpg") for s in txtlist]

In [61]:
folder_export_jpg = "./training data/oma street view/cropped/images/"
folder_export_txt = "./training data/oma street view/cropped/labels/"

for file in txtlist:
    file_jpg = file.replace("txt","jpg")
    img = Image.open(folder_sign + file_jpg)

    with open(folder_sign + file) as labelfile:
        i=0
        for line in labelfile:
            i+=1
            clss = line.split(" ")[0]
            x = float(line.split(" ")[1])
            y = 1-float(line.split(" ")[2])
            w = float(line.split(" ")[3])
            h = float(line.split(" ")[4])

            x1, y1, x2, y2 = bbox_yolo_to_coords_wide(x, y, w, h)

            img2 = img.crop((x1, y2, x2, y1))

            img2.save(folder_export_jpg + "omacomp_" + file_jpg[:-4] + str(i) + ".jpg")
            
            with open(folder_export_txt + "omacomp_" + file_jpg[:-4] + str(i) + ".txt", "a") as txt_file:
                txt_file.write(clss)


### Class vaihto

In [139]:
# VAIHDA CLASS UUTEEN

folder_in = "./Yolo_project/yolov7-custom/data/test/labels_nc27/"
folder_out = "./Yolo_project/yolov7-custom/data/test/labels/"

for txt_file in os.listdir(folder_in):
    with open(folder_in+txt_file) as file:
        for line in file:
            clss_original = int(line.split(" ")[0])
            clss_new = classmap_nc27_to_nc17_num[clss_original]
            if clss_new is not None:
                new_line = str(clss_new) + line[len(str(clss_original)):-1]
                with open(folder_out+txt_file, "a") as new_file:
                    new_file.write(new_line + "\n")

In [12]:
classmap_nc17.values()

dict_values(['kolmio', 'suojatie', 'kävely+pyörä', 'parkki_kielletty', 'alue_parkki_kielletty', 'nuoli_R', 'työmaa', 'kielletty_ajosuunta', 'R_kielletty_käännös', 'L_kielletty_käännös', 'varo_lapsia', 'parkki', 'liikenneYMPYRÄ', 'moottori_AJON_kielletty', 'pysähtyminen_kielletty', 'AJON_kielletty (ympyrä)', 'ETUajo-oikeus', 'kävely-pyörä_Jaettu'])

In [61]:
folder_dir = "./Yolo_project/yolov7-custom/data/train/labels/"
txtlist = []

for x in os.listdir(folder_dir):
    if len(x) < 10:
        txtlist.append(x)

val_txtlist = random.sample(txtlist, 400)
val_txtlist

['475L.txt',
 '263R.txt',
 '462R.txt',
 '3000C.txt',
 '223R.txt',
 '3408L.txt',
 '1602C.txt',
 '2058R.txt',
 '1224L.txt',
 '1425R.txt',
 '1437C.txt',
 '823C.txt',
 '2821C.txt',
 '1889C.txt',
 '456L.txt',
 '1379L.txt',
 '2641C.txt',
 '2932C.txt',
 '2713R.txt',
 '1850R.txt',
 '300C.txt',
 '1734C.txt',
 '45C.txt',
 '2818C.txt',
 '3127C.txt',
 '1405L.txt',
 '1190C.txt',
 '1619L.txt',
 '1683C.txt',
 '1602R.txt',
 '1091L.txt',
 '2828C.txt',
 '201L.txt',
 '549L.txt',
 '2364C.txt',
 '644C.txt',
 '3430R.txt',
 '348C.txt',
 '2322R.txt',
 '215C.txt',
 '2787R.txt',
 '2608R.txt',
 '2390L.txt',
 '3380C.txt',
 '959C.txt',
 '3179C.txt',
 '1727C.txt',
 '557R.txt',
 '1997C.txt',
 '2354C.txt',
 '3034C.txt',
 '3299R.txt',
 '263C.txt',
 '3035L.txt',
 '1486C.txt',
 '1400R.txt',
 '911R.txt',
 '2658C.txt',
 '1919L.txt',
 '2805C.txt',
 '1819C.txt',
 '1224C.txt',
 '559C.txt',
 '677C.txt',
 '463R.txt',
 '1162C.txt',
 '233L.txt',
 '2908L.txt',
 '1254C.txt',
 '3294C.txt',
 '2471R.txt',
 '1404C.txt',
 '380C.txt',
 

In [64]:
val_txtlist_jpg = [s.replace("txt", "jpg") for s in val_txtlist]
val_txtlist_jpg

['475L.jpg',
 '263R.jpg',
 '462R.jpg',
 '3000C.jpg',
 '223R.jpg',
 '3408L.jpg',
 '1602C.jpg',
 '2058R.jpg',
 '1224L.jpg',
 '1425R.jpg',
 '1437C.jpg',
 '823C.jpg',
 '2821C.jpg',
 '1889C.jpg',
 '456L.jpg',
 '1379L.jpg',
 '2641C.jpg',
 '2932C.jpg',
 '2713R.jpg',
 '1850R.jpg',
 '300C.jpg',
 '1734C.jpg',
 '45C.jpg',
 '2818C.jpg',
 '3127C.jpg',
 '1405L.jpg',
 '1190C.jpg',
 '1619L.jpg',
 '1683C.jpg',
 '1602R.jpg',
 '1091L.jpg',
 '2828C.jpg',
 '201L.jpg',
 '549L.jpg',
 '2364C.jpg',
 '644C.jpg',
 '3430R.jpg',
 '348C.jpg',
 '2322R.jpg',
 '215C.jpg',
 '2787R.jpg',
 '2608R.jpg',
 '2390L.jpg',
 '3380C.jpg',
 '959C.jpg',
 '3179C.jpg',
 '1727C.jpg',
 '557R.jpg',
 '1997C.jpg',
 '2354C.jpg',
 '3034C.jpg',
 '3299R.jpg',
 '263C.jpg',
 '3035L.jpg',
 '1486C.jpg',
 '1400R.jpg',
 '911R.jpg',
 '2658C.jpg',
 '1919L.jpg',
 '2805C.jpg',
 '1819C.jpg',
 '1224C.jpg',
 '559C.jpg',
 '677C.jpg',
 '463R.jpg',
 '1162C.jpg',
 '233L.jpg',
 '2908L.jpg',
 '1254C.jpg',
 '3294C.jpg',
 '2471R.jpg',
 '1404C.jpg',
 '380C.jpg',
 

In [74]:
# source_folder = "./Yolo_project/yolov7-custom/data/train/labels/"
# destination_folder = "./Yolo_project/yolov7-custom/data/val/labels/"

# for file_name in val_txtlist:
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     shutil.move(source, destination)

# source_folder = "./Yolo_project/yolov7-custom/data/train/images/"
# destination_folder = "./Yolo_project/yolov7-custom/data/val/images/"

# for file_name in val_txtlist_jpg:
#     source = source_folder + file_name
#     destination = destination_folder + file_name
#     shutil.move(source, destination)

## Detection data

In [3]:
follist = ['suunta1C',
 'suunta1L',
 'suunta1R',
 'suunta2C',
 'suunta2L',
 'suunta2R',]

In [8]:
# source_folder = "./images/fov20/"
# destination_folder = "./Yolo_project/yolov7-custom/detect_data/"

# for fol in follist:
#     for file_name in os.listdir(source_folder+fol):
#         source = source_folder + fol + "/" + file_name
#         destination = destination_folder + fol + "_" + file_name
#         shutil.copy(source, destination)


In [3]:
# source_folder = "./images/fov20/"
# destination_folder = "./Yolo_project/yolov7-custom/detect_data/"

# fol = 'suunta2C'
# for file_name in os.listdir(source_folder+fol):
#     source = source_folder + fol + "/" + file_name
#     destination = destination_folder + fol + "_" + file_name
#     shutil.copy(source, destination)

In [91]:
fol2 = "./Yolo_project/yolov7-custom/data/train/labels/"
fol1 = "./Yolo_project/yolov7-custom/data/train/labels_nc27/"

images = []
labels = []

for file in os.listdir(fol1):
    images.append(file[:-4])

for file in os.listdir(fol2):
    labels.append(file[:-4])

In [92]:
s = set(labels)
temp3 = [x for x in images if x not in s]
temp3

['1002_comp',
 '1003C',
 '1065_comp',
 '1072_comp',
 '1079C',
 '1087C',
 '1101C',
 '1101_comp',
 '1117_comp',
 '1118_comp',
 '1119_comp',
 '1120_comp',
 '1121_comp',
 '1122_comp',
 '1138_comp',
 '1141C',
 '1192C',
 '1204R',
 '1210L',
 '1223L',
 '1231_comp',
 '1239_comp',
 '1240_comp',
 '1244_s1C_shift',
 '1301_comp',
 '1302_comp',
 '1309_comp',
 '1314_s1C_shift',
 '1324_comp',
 '1333_comp',
 '1340R',
 '1344_comp',
 '1356_comp',
 '135_comp',
 '1369_comp',
 '1370_comp',
 '1377_comp',
 '1378_comp',
 '1379_comp',
 '1381_comp',
 '138C',
 '13_comp',
 '140_comp',
 '1416_comp',
 '1417_comp',
 '1440_comp',
 '1441_comp',
 '1442C',
 '1442_comp',
 '1443_comp',
 '144_comp',
 '1469_comp',
 '1474L',
 '1476_comp',
 '1477_comp',
 '1478R',
 '1481_comp',
 '1483_comp',
 '1505C',
 '1516_comp',
 '1525_comp',
 '1526_comp',
 '1527_comp',
 '1540C',
 '1554_comp',
 '1568_comp',
 '1569_comp',
 '1579_comp',
 '1591_comp',
 '1592_comp',
 '1596_comp',
 '1609_comp',
 '1610_comp',
 '1611_comp',
 '1618_comp',
 '1623_com

In [93]:
len(temp3)

550

In [94]:
# for file in temp3:
#     os.remove("./Yolo_project/yolov7-custom/data/train/images/"+file+".jpg")

In [19]:
results = pd.read_excel("./Yolo_project/yolov7-custom/runs/test/exp36_run98/results.xlsx")
results["Class"] = results["Class"].map(fin_classmap)
results

,Class,Images,Labels,P,R,"mAP@,5","mAP@,5:,95:"
0,parkki_kielletty,400,82,0.832,0.847,0.895,0.610
1,L_kielletty_käännös,400,2,0.426,1.000,0.830,0.564
2,parkki,400,58,0.918,0.724,0.809,0.561
3,moottori_AJON_kielletty,400,19,0.812,0.632,0.744,0.561
4,nuoli_R,400,94,0.959,0.872,0.913,0.548
5,suojatie,400,277,0.938,0.814,0.858,0.539
6,kolmio,400,56,1.000,0.747,0.861,0.535
7,kielletty_ajosuunta,400,13,0.842,0.846,0.848,0.526
8,liikenneYMPYRÄ,400,8,0.728,0.750,0.812,0.514
9,all,400,744,0.813,0.733,0.758,0.473
